In [1]:
import tensorflow as tf
import numpy as np

In [2]:
import utils

In [3]:
SAMPLING_RATE = 6
SEQUENCE_LENGTH = 120
DELAY = SAMPLING_RATE * (SEQUENCE_LENGTH + 24 - 1)
BATCH_SIZE = 256

In [4]:
temperature, raw_data = utils.load_data()
num_train_samples = int(0.5 * len(raw_data))


In [5]:
normalized_data, mean, std = utils.normalize_data(raw_data, num_train_samples)


In [6]:
train_dataset, val_dataset, test_dataset = utils.split_data_keras(normalized_data, temperature, sampling_rate = 6, sequence_length=120, batch_size=256)

Notes:
- raw_data -= mean doesn't seem to work?
- why does evaluate_naive_method doesn't give the correct numbers (around 2.9)?

In [7]:
utils.evaluate_naive_method(val_dataset, mean=mean, std=std)

2.4417357485655167

In [8]:
utils.evaluate_naive_method(test_dataset, mean=mean, std=std)

2.6218984517612056

In [9]:
from tensorflow import keras
from tensorflow.keras import layers

In [20]:
inputs = keras.Input(shape=(SEQUENCE_LENGTH, raw_data.shape[-1]))
x = layers.SeparableConv1D(16, 24, activation="relu", padding="same")(inputs)
# x = layers.Dropout(0.5)(x)
x = layers.MaxPooling1D(2)(x)
x = layers.BatchNormalization()(x)
x = layers.SeparableConv1D(32, 12, activation="relu", padding="same")(x)
x = layers.MaxPooling1D(2)(x)
x = layers.BatchNormalization()(x)
x = layers.SeparableConv1D(64, 6, activation="relu", padding="same")(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

In [21]:
def smape(y_true, y_pred):
    raw = tf.math.reduce_mean(tf.math.abs(y_true - y_pred) / (tf.math.abs(y_true) + tf.math.abs(y_pred)))
    return 100*(1-raw)

In [22]:
model.compile(optimizer="adam", loss="mse", metrics=["mae", smape])
print(model.summary())

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 120, 14)]         0         
                                                                 
 separable_conv1d_3 (Separab  (None, 97, 16)           576       
 leConv1D)                                                       
                                                                 
 max_pooling1d_4 (MaxPooling  (None, 48, 16)           0         
 1D)                                                             
                                                                 
 batch_normalization_4 (Batc  (None, 48, 16)           64        
 hNormalization)                                                 
                                                                 
 separable_conv1d_4 (Separab  (None, 37, 32)           736       
 leConv1D)                                                 

In [23]:
callbacks = [keras.callbacks.ModelCheckpoint("jena_dense.keras", save_best_only=True, monitor="val_mae", mode="min")]
history = model.fit(
    train_dataset, epochs=10, validation_data=val_dataset, callbacks=callbacks
)

Epoch 1/10
819/819 [==============================] - 23s 26ms/step - loss: 26.7820 - mae: 3.8973 - smape: 68.1847 - val_loss: 14.1592 - val_mae: 2.9514 - val_smape: 75.3926
Epoch 2/10
819/819 [==============================] - 21s 25ms/step - loss: 12.7054 - mae: 2.8151 - smape: 75.4511 - val_loss: 13.4831 - val_mae: 2.9067 - val_smape: 75.6253
Epoch 3/10
819/819 [==============================] - 20s 25ms/step - loss: 10.7855 - mae: 2.5890 - smape: 76.8347 - val_loss: 13.9736 - val_mae: 2.9646 - val_smape: 75.1413
Epoch 4/10
819/819 [==============================] - 20s 24ms/step - loss: 9.8652 - mae: 2.4744 - smape: 77.6068 - val_loss: 14.5294 - val_mae: 3.0092 - val_smape: 74.9524
Epoch 5/10
819/819 [==============================] - 21s 25ms/step - loss: 9.1948 - mae: 2.3931 - smape: 78.0537 - val_loss: 15.1097 - val_mae: 3.0673 - val_smape: 74.5706
Epoch 6/10
819/819 [==============================] - 22s 27ms/step - loss: 8.6037 - mae: 2.3173 - smape: 78.5367 - val_loss: 15.459

In [28]:
inputs = keras.Input(shape=(SEQUENCE_LENGTH, raw_data.shape[-1]))
x = layers.LSTM(32, return_sequences=True, recurrent_dropout=0.5)(inputs)
x = layers.LSTM(32, recurrent_dropout=0.5)(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(16)(x)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

callbacks = [keras.callbacks.ModelCheckpoint("jena_lstm.keras", save_best_only=True, monitor="val_mae", mode="min")]

model.compile(optimizer="adam", loss="mse", metrics=["mae",smape])

In [30]:
history = model.fit(train_dataset, epochs=10, validation_data = val_dataset, callbacks=callbacks)

Epoch 1/10
  1/819 [..............................] - ETA: 18:16 - loss: 10.3913 - mae: 2.5252 - smape: 79.6664

KeyboardInterrupt: 